In [4]:
from copy import deepcopy
import numpy as np

In [8]:
def attempted_move(current_position, direction):
    if direction == '^':
        return current_position + np.array([-1,0])
    elif direction == '>':
        return current_position + np.array([0,1])
    elif direction == '<':
        return current_position + np.array([0,-1])
    elif direction == 'v':
        return current_position + np.array([1,0])

class Day15:
    def __init__(self, fname):
        with open(fname) as f:
            all = f.read()
        start_map, movements = all.split('\n\n')
        start_map = start_map.split('\n')
        start_map = [list(row) for row in start_map]
        movements = [mv for mv in movements if mv != "\n"]
        self.start_map = np.array(start_map)
        self.current_map = deepcopy(self.start_map)
        self.movements = movements
        self.start_location = np.concatenate(np.where(self.current_map == '@'))
        self.current_location = deepcopy(self.start_location)
        self.remaining_movements = deepcopy(movements)

    def has_space_to_move(self, attempted_location, direction):
        next_attempt = attempted_location
        while True:
            next_attempt = attempted_move(next_attempt, direction)
            if self.current_map[tuple(next_attempt)] == '.':
                return next_attempt
            elif self.current_map[tuple(next_attempt)] == '#':
                return False


    def move_once(self):
        direction = self.remaining_movements.pop(0)
        attempted_location = attempted_move(self.current_location, direction)
        currently_in_that_location = self.current_map[tuple(attempted_location)]
        if currently_in_that_location == '.':
            # robot just moves into that spot, and its current location becomes a '.'
            self.current_map[tuple(self.current_location)] = '.'
            self.current_map[tuple(attempted_location)] = '@'
            self.current_location = attempted_location
        elif currently_in_that_location == '#':
            pass
            # nothing happens if it hits a wall.
        elif currently_in_that_location == 'O':
            end_space = self.has_space_to_move(attempted_location, direction)
            if end_space is not False:
                self.current_map[tuple(end_space)] = 'O'
                self.current_map[tuple(self.current_location)] = '.'
                self.current_map[tuple(attempted_location)] = '@'
                self.current_location = attempted_location
            else:
                pass
    
    def move_all(self):
        while len(self.remaining_movements) > 0:
            self.move_once()
        boxes = np.where(self.current_map == 'O')
        total = np.sum(boxes[0] * 100) + np.sum(boxes[1])
        return total






In [123]:
class Day15Supersize(Day15):
    def __init__(self, fname):
        super().__init__(fname)
        self.start_location *= np.array([1, 2])
        self.current_location *= np.array([1, 2])
        original_dimensions = self.start_map.shape
        mapping = {'.' : ['.','.'], '#' : ['#','#'], 'O' : ['[',']'], '@' : ['@', '.']}
        new_map = np.array(
            [mapping[elem] for row in self.current_map for elem in row]
        )
        self.start_map = new_map.reshape(original_dimensions[0], original_dimensions[1] * 2)
        self.current_map = deepcopy(self.start_map)

    def has_space_to_move_leftright(self, attempted_location, direction):
        next_attempt = attempted_location
        while True:
            next_attempt = attempted_move(next_attempt, direction)
            if self.current_map[tuple(next_attempt)] == '.':
                return next_attempt
            elif self.current_map[tuple(next_attempt)] == '#':
                return False
            
    def shove_box_updown(self, list_of_locations, full_list, direction):
        # change = np.array([-1, 0]) if direction == '^' else np.array([1, 0])
        new_list = []
        for location in list_of_locations:
            next_location = attempted_move(location, direction)
            currently_there = self.current_map[tuple(next_location)]
            if currently_there == '#':
                return False
            elif currently_there == '.':
                pass
            elif currently_there == '[':
                if tuple(list(next_location)) in [tuple(list(loc)) for loc in new_list]:
                    continue
                new_list.append(next_location)
                new_list.append(next_location + np.array([0, 1]))
            elif currently_there == ']':
                if tuple(list(next_location)) in [tuple(list(loc)) for loc in new_list]:
                    continue
                new_list.append(next_location)
                new_list.append(next_location + np.array([0, -1]))
        if len(new_list) == 0:
            return full_list
        else:
            full_list.append(new_list)
            return self.shove_box_updown(new_list, full_list, direction)

    def move_box_updown(self, attempted_location, currently_in_that_location, direction):
        change = np.array([-1, 0]) if direction == '^' else np.array([1, 0])
        other_side_of_box = attempted_location + np.array([0, 1]) if currently_in_that_location == '[' else  attempted_location + np.array([0, -1])
        spaces_to_move = [attempted_location, other_side_of_box]
        shove_box_result = self.shove_box_updown(spaces_to_move, [spaces_to_move], direction)
        if shove_box_result is not False:
            for i in range(len(shove_box_result)):
                this_list = shove_box_result.pop(-1)
                for item in this_list:
                    self.current_map[tuple(item + change)] = self.current_map[tuple(item)]
                    self.current_map[tuple(item)] = '.'
            self.current_map[tuple(self.current_location)] = '.'
            self.current_map[tuple(attempted_location)] = '@'
            self.current_location = attempted_location
        else:
            return

    def move_box_left_right(self, attempted_location, direction):
        mapping = {'[' : ']', ']' : '['}
        end_space = self.has_space_to_move_leftright(attempted_location, direction)
        if end_space is not False:
            start = attempted_location # Starting index
            end = end_space  # Ending index (exclusive along columns)
            if direction == '<':
                self.current_map[end[0], (end[1] + 1):(start[1] + 1)] = np.array(
                    [mapping[elem] for elem in self.current_map[end[0], (end[1] + 1):(start[1] + 1)]]
                )
                self.current_map[tuple(end_space)] = '['
            elif direction == '>':
                self.current_map[start[0], start[1]:end[1]] = np.array(
                    [mapping[elem] for elem in self.current_map[start[0], start[1]:end[1]]]
                )
                self.current_map[tuple(end_space)] = ']'
            self.current_map[tuple(self.current_location)] = '.'
            self.current_map[tuple(attempted_location)] = '@'
            self.current_location = attempted_location
        else:
            return

    def move_once(self):
        direction = self.remaining_movements.pop(0)
        attempted_location = attempted_move(self.current_location, direction)
        currently_in_that_location = self.current_map[tuple(attempted_location)]
        if currently_in_that_location == '.':
            # robot just moves into that spot, and its current location becomes a '.'
            self.current_map[tuple(self.current_location)] = '.'
            self.current_map[tuple(attempted_location)] = '@'
            self.current_location = attempted_location
        elif currently_in_that_location == '#':
            pass
            # nothing happens if it hits a wall.
        elif currently_in_that_location in ['[', ']']:
            if direction in ['<', '>']:
                self.move_box_left_right(attempted_location, direction)
            if direction in ['^', 'v']:
                self.move_box_updown(attempted_location, currently_in_that_location, direction)
    
    def move_all(self):
        while len(self.remaining_movements) > 0:
            self.move_once()
        boxes = np.where(self.current_map == '[')
        total = np.sum(boxes[0] * 100) + np.sum(boxes[1])
        return total


In [127]:
day15_test = Day15('data/test.txt')
print(day15_test.move_all())
day15_test_Supersize = Day15Supersize('data/test.txt')
print(day15_test_Supersize.move_all())


10092
9021


In [126]:
day15 = Day15('data/input.txt')
print(day15.move_all())
day15_Supersize = Day15Supersize('data/input.txt')
print(day15_Supersize.move_all())



1479679
1509780
